In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

HR_mem (generic function with 1 method)

In [2]:
using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using CairoMakie, GLMakie
using LinearAlgebra

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305;

In [4]:
t = 5000; tt = 1000; tstep = 0.0001; tspan = [time, tt]
integ_set = (alg = Vern9(), adaptive = true, abstol = 1e-15, reltol = 1e-15);

In [20]:
I0 = -1.7075; U0 = 0.26504556977786675
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [11.325905642223786, 0.6594706953104683, 0.4863175026548461];

In [21]:
ds = CoupledODEs(TM, u0, p, diffeq = integ_set)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  TM
 ODE solver:    Vern9
 ODE kwargs:    (adaptive = true, abstol = 1.0e-15, reltol = 1.0e-15)
 parameters:    [1.58, 0.013, 0.07993, 3.3, 3.07, 0.75, 0.4, 0.26504556977786675, 0.305, 0.3, -1.7075]
 time:          0.0
 state:         [11.325905642223786, 0.6594706953104683, 0.4863175026548461]


In [22]:
fp, ei, _ = fixedpoints(ds, box, jacob_TM_)

index_fp = 1
index_vec = 1
ϵ = 1e-9

Jac = jacob_TM_(fp[index_fp], p, 0)
eivecs = eigvecs(Jac)
shift =  fp[index_fp] + real(eivecs[:, index_vec])*ϵ
println("fixed point with shift along stable manifold: $(shift)")

fixed point with shift along stable manifold: [8.34447593715034, 0.7384982011908008, 0.4383158368374711]


In [23]:
sol, trange = trajectory(ds, t, Ttr = tt, Δt = tstep)
println("Length sol: $(length(sol))");

Length sol: 50000001


In [24]:
lb_size_ts = 30; tck_size_ts = 25
width = 2000; height = 800;

In [10]:
function global_max(timesereis, ϵ = 1e-2)

    tr, trange = timeseries
    glob_maxs, trange_glob_maxs = Float64[], Float64[]
    real_global_max =floor(maximum(tr), digits = 3)
    idxs = Int64[]
    
    for idx in range(2, length(tr) - 1, step = 1)
        if tr[idx-1] < tr[idx] > tr[idx+1]
            if abs(tr[idx] -real_global_max) < ϵ
                push!(glob_maxs, tr[idx])
                push!(trange_glob_maxs, trange[idx])
                push!(idxs, idx)
            end
        end
    end

    return glob_maxs,trange_glob_maxs, idxs
end
function interval_turns(timeseries, index, fp)

    tr, trange = timeseries
    
    flag = false
    timelabel = 0
    
    while flag == false
        index+=1
        if sol[index, 3] <= fp[1][3]
            timelabel = index - 1
            break
        end
    end
    return timelabel
end

function local_max(sol, index_global_max, timelabel)
   
    index = index_global_max
    local_maxs, trange_local_maxs = Float64[], Float64[]
    idxs = Int64[]
    while index <= timelabel

        if sol[index-1, 1] < sol[index, 1] > sol[index+1, 1]
            push!(local_maxs, sol[index, 1])
            push!(trange_local_maxs, trange[index])
            push!(idxs, index)
        end

        index+=1
    end
    index-=1

    return  local_maxs,trange_local_maxs, idxs
end

local_max (generic function with 1 method)

In [25]:
timeseries = [sol[:, 3], trange]
glob_maxs , trange_glob_maxs, indexs_global_max = global_max(timeseries, 1e-2)

([0.5881708417994874, 0.5881708397528954, 0.5881708432520552, 0.5881708414998799, 0.5881708401776423, 0.5881708433010471, 0.5881708411734268, 0.5881708405755056, 0.5881708433231809, 0.5881708408201339  …  0.5881708428628657, 0.58817084243959, 0.5881708385536379, 0.5881708430069488, 0.5881708422081481, 0.5881708390466607, 0.5881708431241597, 0.5881708419498554, 0.5881708395127861, 0.5881708432145005], [1000.5232, 1005.6061, 1010.6889, 1015.7717, 1020.8546, 1025.9374, 1031.0202, 1036.1031, 1041.1859, 1046.2687  …  5951.2037, 5956.2865, 5961.3694, 5966.4522, 5971.535, 5976.6179, 5981.7007, 5986.7835, 5991.8664, 5996.9492], [5233, 56062, 106890, 157718, 208547, 259375, 310203, 361032, 411860, 462688  …  49512038, 49562866, 49613695, 49664523, 49715351, 49766180, 49817008, 49867836, 49918665, 49969493])

In [26]:
count_global_max = length(indexs_global_max)-1
range_global_max = range(1, count_global_max, step = 1)
max_count_turns = Int64[]

Int64[]

In [27]:
for i in range_global_max
    index_global_max = indexs_global_max[i]
    timelabel = interval_turns(timeseries, index_global_max, fp)
    local_maxs,trange_local_maxs, idxs = local_max(sol, index_global_max, timelabel)
    push!(max_count_turns, length(local_maxs) - 1)
end

In [28]:
maximum(max_count_turns)

4

In [29]:
number_global_max = length(indexs_global_max)-1
index_global_max = indexs_global_max[number_global_max]
timelabel = interval_turns(timeseries, index_global_max, fp)

49938345

In [18]:
local_maxs,trange_local_maxs, idxs = local_max(sol, index_global_max, timelabel)
println("Number of turns: $(length(local_maxs) - 1)")

Number of turns: 4


In [ ]:
"""f = Figure(resolution = (900, 900))
ax = Axis(f[1, 1])

lines!(ax, sol[index_global_max:timelabel, 3], sol[index_global_max:timelabel, 2], linewidth = 0.5 )

for idx in idxs    
    scatter!(ax, sol[idx, 3], sol[idx, 2], color = :black, markersize = 5 )
end

vlines!(ax, fp[1][3], color = :blue, linewidth = 0.5)

display(GLMakie.Screen(), f)"""

In [ ]:
"""ts, tf = index_global_max, timelabel+48000

f = Figure(resolution = (width, height))
axisE = Axis(f[1, 1], xlabel = L"$time$", ylabel = L"$E$",
            xlabelsize = lb_size_ts, ylabelsize = lb_size_ts,
            xticklabelsize = tck_size_ts, yticklabelsize = tck_size_ts,
            xgridvisible = false, ygridvisible = false)

axisy = Axis(f[2, 1], xlabel = L"$time$", ylabel = L"$y$",
            xlabelsize = lb_size_ts, ylabelsize = lb_size_ts,
            xticklabelsize = tck_size_ts, yticklabelsize = tck_size_ts,
            xgridvisible = false, ygridvisible = false)

lines!(axisE, trange[ts:tf], sol[ts:tf, 1], linewidth = 1.5, color = :deeppink)
scatter!(axisE, trange_local_maxs, local_maxs)
vlines!(axisE,  trange[timelabel, 1])

lines!(axisy, trange[ts:tf], sol[ts:tf, 3], linewidth = 1.5, color = :deeppink)
vlines!(axisy,  trange[timelabel, 1])


hlines!(axisy, fp[1][3], color = :green)
hlines!(axisE, fp[1][1], color = :green)

display(GLMakie.Screen(), f)
#display(f)"""

In [30]:
ts, tf = index_global_max, timelabel
#tscon, tfcon = timelabel, timelabel + 100
idx, idy, idz = 2, 3, 1
dis  = 60;

f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)
    
scale!(ax.scene, 50, 50, 1)

lines!(sol[ts:tf, idx], sol[ts:tf, idy], sol[ts:tf, idz], linewidth = 1.0, color = :black)

lines!(sol[index_global_max:idxs[2], idx], sol[index_global_max:idxs[2], idy], sol[index_global_max:idxs[2], idz], linewidth = 1.0, color = :green)
lines!(sol[idxs[2]:idxs[3], idx], sol[idxs[2]:idxs[3], idy], sol[idxs[2]:idxs[3], idz], linewidth = 1.0, color = :orange)
lines!(sol[idxs[3]:idxs[4], idx], sol[idxs[3]:idxs[4], idy], sol[idxs[3]:idxs[4], idz], linewidth = 1.0, color = :deeppink)
lines!(sol[idxs[4]:idxs[5], idx], sol[idxs[4]:idxs[5], idy], sol[idxs[4]:idxs[5], idz], linewidth = 1.0, color = :lime)
#lines!(sol[idxs[5]:idxs[6], idx], sol[idxs[5]:idxs[6], idy], sol[idxs[5]:idxs[6], idz], linewidth = 1.0, color = :magenta)




for (i, ep) in enumerate(fp)
    scatter!(ep[idx], ep[idy], ep[idz], color = :blue, markersize = 8)
end

scatter!(ax, sol[indexs_global_max[number_global_max], idx], sol[indexs_global_max[number_global_max], idy], sol[indexs_global_max[number_global_max], idz], color = :green, markersize = 8)

for id in idxs    
    scatter!(ax, sol[id, idx], sol[id, idy], sol[id, idz], color = :red, markersize = 8 )
end

display(GLMakie.Screen(), f)

GLMakie.Screen(...)